# Time Series Forecasting mit rekurrenten neuronalen Netzen
Dieses Notebook entsteht im Rahmen des Moduls `Deep Learning` an der Fachhochschule Südwestfalen. Es beschäftigt sich mit dem Thema `Time Series Forecasting` und untersucht die Anwendung von rekurrenten neuronalen Netzen (RNN) auf Zeitreihendaten....

## 1. Einleitung

## 2. Was ist Time Series Forecasting (Zeitreihenvorhersage)?

Christian

Definition gemäß https://www.studysmarter.de/studium/mathematik-studium/statistik-studium/zeitreihenprognose/ (27.12.2024):<br>
**Zeitreihenprognose** ist eine statistische Methode zur Vorhersage zukünftiger Ereignisse oder Werte, indem historische Zeitreihendaten analysiert und Muster erkannt werden.

Beispiele:<br>
- Wetterdaten für zukünftiges Wetter (https://www.studysmarter.de/studium/mathematik-studium/statistik-studium/zeitreihenprognose/)
- Aktienkurse für zukünftige Aktienverläufe
- Passagierzahlen für zukünftige Passagiervorhersagen (https://data-science-crashkurs.de/chapters/kapitel_09.html)

Die Seite https://www.emft.fraunhofer.de/de/kompetenzen/systemloesungen-ki/ki-algorithmen-zeitreihenanalyse.html nennt zudem noch folgende mögliche Beispiele:
- Industrie 4.0 mit IIoT (Industrial Internet of Things)
- Logistik
- Finanzen
- Stromnetze
- Internetverkehr
- Umweltmonitoring
- Gesundheitsdaten

Außerdem werden auf der Seite typische Methoden der künstlichen Intelligenz inklusive grafiscgenannt, wie z.B.: Regression, Klassifikation, Anomalieerkennung, Vorhersage

Grafisches Beispiel einer Klassifikation:<br>
<img style = 'border: 5px solid #555' src="Pictures/classification-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Klassifikation"> 
<br>
Grafisches Beispiel einer Anomalieerkennung:<br>
<img style = 'border: 5px solid #555' src="Pictures/anomaly-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Anomalieerkennung">
<br>
Grafisches Beispiel einer Anomalieerkennung:<br>
<img style = 'border: 5px solid #555' src="Pictures/forecasting-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Vorhersage">

## 3. Was sind recurrent neural networks (Rekurrente neuronale Netzwerke)?

In [ ]:
Tim

## 4. Datensatz

## 5. Implementierung eines RNN

## 6. Aktuelle Modelle
Beispielmodelle Goldpreis oder Aktienpreis

## 7. Vergleich der Modelle

## 8. Fazit
Eigenes Modell vs. verglichene Standardmodelle.